In [9]:
import os
import glob

In [8]:
os.path.join('E:/s', 'abd', 'abc', 'd')

'E:/s\\abd\\abc\\d'

In [25]:
os.listdir("E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release\\Session1\\sentences\\wav\\Ses01F_impro01\\")

['Ses01F_impro01_F000.wav',
 'Ses01F_impro01_F001.wav',
 'Ses01F_impro01_F002.wav',
 'Ses01F_impro01_F003.wav',
 'Ses01F_impro01_F004.wav',
 'Ses01F_impro01_F005.wav',
 'Ses01F_impro01_F006.wav',
 'Ses01F_impro01_F007.wav',
 'Ses01F_impro01_F008.wav',
 'Ses01F_impro01_F009.wav',
 'Ses01F_impro01_F010.wav',
 'Ses01F_impro01_F011.wav',
 'Ses01F_impro01_F012.wav',
 'Ses01F_impro01_F013.wav',
 'Ses01F_impro01_F014.wav',
 'Ses01F_impro01_F015.wav',
 'Ses01F_impro01_M000.wav',
 'Ses01F_impro01_M001.wav',
 'Ses01F_impro01_M002.wav',
 'Ses01F_impro01_M003.wav',
 'Ses01F_impro01_M004.wav',
 'Ses01F_impro01_M005.wav',
 'Ses01F_impro01_M006.wav',
 'Ses01F_impro01_M007.wav',
 'Ses01F_impro01_M008.wav',
 'Ses01F_impro01_M009.wav',
 'Ses01F_impro01_M010.wav',
 'Ses01F_impro01_M011.wav',
 'Ses01F_impro01_M012.wav',
 'Ses01F_impro01_M013.wav']

In [18]:
"asbdiasf.wav"[:-4]

'asbdiasf'

In [19]:
a = []
b = [1, 2, 3]

In [21]:
a + b + b

[1, 2, 3, 1, 2, 3]

In [2]:
from data.mfcc_extraction import IEMOCAP_Dataset

ROOT = "E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release"
iemocap = IEMOCAP_Dataset(root=ROOT)

In [2]:
iemocap.utterance_paths[0]

'E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release\\Session1\\sentences\\wav\\Ses01F_impro01\\Ses01F_impro01_F000.wav'

In [3]:
len(iemocap.utterance_list)

10039

In [3]:
len(iemocap.label_map.items())

10039

In [9]:
import wave
import numpy as np
import python_speech_features as psf

PATH = iemocap.utterance_paths[1]
with wave.open(PATH, 'r') as f:
    nchannels, sampwidth, framerate, wav_length = f.getparams()[:4]
    data = np.frombuffer(f.readframes(wav_length), dtype=np.short)
    print(data)
    time = np.arange(0,wav_length) * (1.0/framerate)
    print(time)

[ 31 -31 -23 ... -15 -11 -42]
[0.0000000e+00 6.2500000e-05 1.2500000e-04 ... 1.3822500e+00 1.3823125e+00
 1.3823750e+00]


In [10]:
mel_spec = psf.logfbank(data, framerate, nfilt=40)
delta1 = psf.delta(mel_spec, 2)
delta2 = psf.delta(delta1, 2)
display(mel_spec.shape)
display(delta1.shape)
display(delta2.shape)

(137, 40)

(137, 40)

(137, 40)

In [19]:
np.stack((mel_spec, delta1, delta2), axis=1).shape

(137, 3, 40)

In [21]:
'Ses01F_impro01_F000'[:5]

'Ses01'

In [24]:
import numpy as np

a = np.random.randint(5, size=(3,3,3))
b = 1

np.array((b, a), dtype=object)

array([1, array([[[2, 4, 2],
                  [2, 2, 4],
                  [3, 3, 4]],

                 [[4, 3, 3],
                  [0, 3, 0],
                  [1, 1, 0]],

                 [[4, 3, 2],
                  [2, 1, 2],
                  [2, 2, 4]]])], dtype=object)

In [25]:
sample = np.load("E:\\Download\\IEMOCAP_full_release_withoutVideos\\data\\train\\Ses01F_impro01_F000.npy", allow_pickle=True)

In [27]:
sample[1].shape

(194, 3, 40)